In [92]:
import numpy as np

In [93]:
from sklearn.datasets import fetch_20newsgroups_vectorized
X, y = fetch_20newsgroups_vectorized(subset='train', 
                                     return_X_y=True, 
                                     remove=('headers', 'footers'))

In [94]:
X.shape, y.shape

((11314, 114751), (11314,))

# Préparation des données

Dans la suite, on va garder les 5 classes les plus représentées parmi les 1000 premiers exemples de `X`. Les exemples associés autres autres classes sont écartés. On obtient finalement un échantillon que vous appelez `X_g` de 306 exemples.  

In [95]:
nb_classes = 5
n_exemples = 1000

**Question**  Utiliser [np.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) avec l'argument `return_counts` pour identifier dans `labels` les classes que vous gardez.

In [96]:
a = np.unique(y[:n_exemples] , return_counts=True)

In [97]:
# get 5 argmaxes 

idx = np.argsort(a)
idx

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [18, 19,  3, 16, 11, 12,  7, 14,  4,  8,  0,  9,  5, 17,  2, 15,
         6, 10, 13,  1]])

In [98]:
classes = idx[1 , -nb_classes:]


In [99]:
# get indexes of the elements in y that are in classes
idx = np.isin(y[:n_exemples] , classes)
X_g = X[:n_exemples][idx]
y_g = y[:n_exemples][idx]

In [100]:
print(X_g.shape)
print(y_g.shape)


(306, 114751)
(306,)


Si vous n'avez pas réussi à faire cela, vous pouvez charger les données dans `pb.csv.xz`. (c'est un fichier compressé que panda sait lire).

# Encodage des classes

Dans la méthode de Zhou et al 2003, nous avons regardé l'identité entre un calcul itératif et une version analytique de ce calcul. Mais nous n'avons pas réellement défini une méthode de classification. En fait le problème de classification est résolu en prenant une fonction $F$ qui encode le problème de décision associé à la classification.

La méthode s'applique généralement aux problèmes multiclasses, avec un nombre $c$ de classes. 

Pour chaque noeud, la fonction $F$ associe un score à chaque classe possible dans $[0,\dots,c-1]$. Alors $F(i, k)$ est le score associé à la classe $k$ pour le noeud $i$. On fera une décision en prenant le score maximal associé à chaque noeud :

\begin{equation}
\mathop{\mathrm {argmax}}_{0\leq k< c} F(i,k)\hspace{5cm}(1)
\end{equation}

En fait $F$ est l'encodage de `y_g` par un `OneHotEncoder`.

**Question** Calculer `F_star` qui est l'encodage des classes à l'aide de cette méthode.

In [101]:
# one hot encoding of y_g sk learn
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
f_star = enc.fit_transform(y_g.reshape(-1, 1)).toarray()
f_star

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

Dans ce problème semi-supervisé, la fonction $F$ est initialisée en mettant $F(i, k)=1$ pour tous les noeuds dont on connait l'étiquette associée $k$ et 0 partout ailleurs.  

**Question** Définir `F`

In [102]:
nb_labeles = 25

In [103]:
# f is a copy of f_star in the first nb_labels columns , else 0
f = f_star.copy()
f[nb_labeles:, :] = 0
f

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

# Label spreading

In [104]:
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.neighbors import kneighbors_graph

**Question** Définir deux matrices `M_rbf` et `M_knn` pour calculer deux noyaux de similarité à partir 
- d'un noyau gaussien (on prendra $\gamma=20$), et 
- d'un $k$ plus proches voisins (on prendra $k=3$). 

In [105]:
M_rbf = rbf_kernel(X_g, gamma=20)
M_knn = kneighbors_graph(X_g, n_neighbors = 3).toarray()

**Question** Définir une fonction `getS` qui prend en argument une matrice noyau et calcule la matrice $S$ associée à la méthode de Zhou et al 2003. NB: la diagonale de la matrice noyau doit être nulle. 

In [125]:

def getS(M):
    M = M - np.diag(np.diag(M))
    D = np.diag(M.sum(axis=1))
    D_inv = np.linalg.inv(D)
    D= np.sqrt(D_inv)
    return  D@M@D

In [126]:
np.diag(np.diag(M_rbf)).shape , M_rbf.shape

((306, 306), (306, 306))

In [127]:
getS(M_knn)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [128]:
getS(M_rbf)

array([[0.00000000e+00, 2.84035570e-05, 1.36480268e-06, ...,
        5.10162555e-07, 3.77380198e-10, 3.05327092e-05],
       [2.84035570e-05, 0.00000000e+00, 2.12237007e-05, ...,
        2.56360098e-06, 2.65616057e-09, 1.66986664e-03],
       [1.36480268e-06, 2.12237007e-05, 0.00000000e+00, ...,
        5.16146095e-05, 4.16255701e-09, 4.38224277e-05],
       ...,
       [5.10162555e-07, 2.56360098e-06, 5.16146095e-05, ...,
        0.00000000e+00, 5.07259105e-09, 3.97522585e-07],
       [3.77380198e-10, 2.65616057e-09, 4.16255701e-09, ...,
        5.07259105e-09, 0.00000000e+00, 1.96244383e-09],
       [3.05327092e-05, 1.66986664e-03, 4.38224277e-05, ...,
        3.97522585e-07, 1.96244383e-09, 0.00000000e+00]])

**Question** écrire une une fonction `zhou_iter` qui prend en argument, 
- une matrice noyau `M`, 
- une matrice `F` qui encode les classes (avec une partie supervisée et une partie non supervisée comme fait précédemment),
- une liste de classes (comme `labels`) qui correspond à chaque colonne de `F`,
- une valeur de $\alpha$ (0.2 par défaut) et  
- `max_iter` correspondant nombre maximal d'itérations. 

La fonction retourne le vecteur des classes correspondant au maximum définit par l'équation (1) ci-dessus. 

In [129]:
# zhou iter , matrice M , f encode classes , list of labels , alpha , max_iter 
def zhou_iter(M, f, labels, alpha, max_iter):
    S = getS(M)
    f_star_old = f.copy()
    f_star = alpha * S @ f_star_old + (1 - alpha) * f
    for i in range(max_iter-1 ):
        
        if np.allclose(f_star, f_star_old):
            break
        else : 
            f_star_helper = f_star.copy()
            f_star = alpha * S @ f_star_old + (1 - alpha) * f
            f_star_old = f_star_helper.copy()
    print(i)
    return labels[np.argmax(f_star , axis=1)]

In [130]:
sol1 = zhou_iter(M_rbf, f, classes, 0.2, 1000)

1


**Question** Écrire maintenant une fonction `zhou_analytique` qui fait ce calcul mais à l'aide de l'expression analytique. Les paramètres sont identiques (hormis `max_iter`). La valeur retournée est identique.

In [131]:
# analytique version 

def zhou_analytique(M, F, labels, alpha=0.2):
    return labels[np.argmax(np.linalg.inv(np.eye(M.shape[0])- alpha*getS(M))@F, axis=1)]

In [132]:
sol2 = zhou_analytique(M_rbf, f, classes, 0.2)

**Question** Calculer l'erreur en classification avec vos implémentations de la méthode de Zhou.

In [133]:
sol1 

array([10, 10,  1, 15, 15, 15, 13, 15, 10, 10, 10,  6, 10,  1,  6, 10, 13,
       15,  1, 13, 15, 13, 10, 15,  1, 10, 10, 13, 15, 15, 13, 10,  1, 10,
       10, 13,  1, 13, 10,  1, 13,  1, 10, 15,  1, 10, 10,  1, 13,  1, 15,
        1, 10,  1,  1,  1,  1, 10, 13, 13,  1, 15, 13, 13,  1, 15, 15,  1,
        1, 15, 13, 10,  1,  1, 15,  1,  1,  1, 15, 15, 10, 10, 10,  1,  1,
       13, 10, 15, 10, 13, 15,  1, 15, 10,  1, 10,  1, 13, 15, 10, 13, 10,
       10, 13, 13,  6,  1,  1, 13, 10, 10, 15,  1,  1, 13, 13,  1, 13, 13,
       10, 13,  1, 10, 10, 10, 13, 10,  1,  1,  1, 13, 10, 13, 10, 15, 10,
       13, 10, 10,  1, 10, 10, 13, 15,  1, 13, 10, 15, 15, 15, 10,  1,  1,
       10, 10, 10, 13, 13, 13,  1,  1,  1, 13, 10, 13, 15,  1, 10, 15, 10,
        1, 13, 15,  1,  1, 13, 10, 10,  1, 13, 15, 15, 13, 13, 10, 13, 10,
       15, 15,  1, 10, 10, 15, 10,  1,  1,  6,  1, 10, 10, 13, 10, 15, 10,
       13, 10,  1, 13, 10, 15, 10,  1, 10, 13, 13, 15, 10,  1, 15,  6, 13,
       15, 15, 10, 15, 10

In [134]:
sol2 

array([10, 10,  1, 15, 15, 15, 13, 15, 10, 10, 10,  6, 10,  1,  6, 10, 13,
       15,  1, 13, 15, 13, 10, 15,  1, 10, 10, 13, 15, 15, 13, 10,  1, 10,
       10, 13,  1,  1, 10,  1,  1,  1, 10, 15,  1, 10, 10,  1, 13,  1, 15,
        1, 10,  1,  1,  1,  1, 10, 13, 13,  1, 15, 13, 13,  1, 15, 15,  1,
        1, 15, 13, 10,  1,  1,  1,  1,  1,  1, 15, 15, 10, 10,  1,  1,  1,
        1, 10, 15, 10, 13, 15,  1, 15, 10,  1, 10,  1, 13, 15, 10, 13, 10,
       10, 13, 13, 15,  1,  1, 13, 10, 10, 15,  1,  1, 13, 13,  1, 13, 13,
       10, 13,  1, 10, 10, 10, 13, 10,  1,  1,  1, 13, 10, 13, 10, 15, 10,
       10, 10, 10,  1, 10, 10, 13, 15,  1,  1,  1, 15, 15,  1, 10,  1,  1,
       10, 10, 10, 13, 13, 13,  1,  1,  1, 13, 10, 13, 15,  1, 10, 15, 10,
        1, 13, 15,  1,  1, 13, 10, 10,  1, 13, 15, 15,  1, 13, 10, 13, 10,
       15, 15,  1, 10, 10, 15, 10,  1,  1,  6,  1, 10, 10, 13, 10,  1, 10,
       10, 10,  1, 13, 10, 15, 10,  1, 10, 13,  1, 15, 10,  1, 15,  6, 13,
       15, 15, 10, 15, 10

In [135]:
accuracy_score(y_g , sol2 )

0.30392156862745096

In [136]:
accuracy_score(y_g , sol1 )

0.31699346405228757

# Avec Sklearn

**Question** Refaire la même chose avec sklearn : calculer l'erreur de classification avec la méthode de Zhou implantée dans sklearn. (NB: le résultat peut être légèrement différent).

In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score


In [ ]:
label_prop_model = LabelPropagation(kernel='rbf', gamma=20, max_iter=1000)
label_prop_model.fit(X_g[:nb_labeles], y_g[:nb_labeles])
y_pred = label_prop_model.predict(X_g[nb_labeles:])
print("accuracy score : ", accuracy_score(y_g[nb_labeles:], y_pred))


accuracy score :  0.2918149466192171
